# VectrixDB Tiers & Modes

Understanding the different storage tiers and search modes available in VectrixDB.

## Storage Tiers

VectrixDB offers 4 storage tiers:
- **dense**: Vector embeddings only (fastest, smallest)
- **hybrid**: Vector + BM25 sparse (balanced)
- **ultimate**: Vector + BM25 + ColBERT late interaction (best quality)
- **graph**: Ultimate + knowledge graph (for GraphRAG)

In [1]:
from vectrixdb import Vectrix

# Sample data
texts = [
    "Machine learning is a subset of artificial intelligence.",
    "Deep learning uses neural networks with many layers.",
    "Natural language processing handles text and speech.",
    "Computer vision enables machines to interpret images.",
    "Reinforcement learning trains agents through rewards."
]

### Dense Tier
Fastest and smallest. Uses only vector embeddings.

In [2]:
db_dense = Vectrix("test_dense", language="en")
db_dense.add(texts)

results = db_dense.search("AI and machine learning")
print("Dense tier results:")
for r in results:
    print(f"  {r.score:.4f}: {r.text[:50]}...")

Dense tier results:
  0.0180: Machine learning is a subset of artificial intelli...
  0.0177: Computer vision enables machines to interpret imag...
  0.0173: Deep learning uses neural networks with many layer...
  0.0172: Reinforcement learning trains agents through rewar...
  0.0078: Natural language processing handles text and speec...


### Hybrid Tier
Combines vector search with BM25 keyword matching.

In [3]:
db_hybrid = Vectrix("test_hybrid", tier="hybrid", language="en")
db_hybrid.add(texts)

results = db_hybrid.search("neural networks", mode="hybrid")
print("Hybrid tier results:")
for r in results:
    print(f"  {r.score:.4f}: {r.text[:50]}...")

Hybrid tier results:
  0.0180: Deep learning uses neural networks with many layer...
  0.0081: Machine learning is a subset of artificial intelli...
  0.0079: Computer vision enables machines to interpret imag...
  0.0078: Natural language processing handles text and speec...
  0.0077: Reinforcement learning trains agents through rewar...


### Ultimate Tier
Adds ColBERT late interaction for maximum precision.

In [4]:
db_ultimate = Vectrix("test_ultimate", tier="ultimate", language="en")
db_ultimate.add(texts)

results = db_ultimate.search("text processing", mode="ultimate")
print("Ultimate tier results:")
for r in results:
    print(f"  {r.score:.4f}: {r.text[:50]}...")

C:\Users\kwadw.DESKTOP-T9BSTPE\anaconda3\Lib\site-packages\torch\cuda\__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


Ultimate tier results:
  -9.1796: Natural language processing handles text and speec...
  -9.1796: Deep learning uses neural networks with many layer...
  -9.1796: Computer vision enables machines to interpret imag...
  -9.1796: Machine learning is a subset of artificial intelli...
  -9.1796: Reinforcement learning trains agents through rewar...


## Search Modes

Different modes available depending on your tier:
- **dense**: Pure vector similarity
- **sparse**: BM25 keyword matching
- **hybrid**: Combined dense + sparse
- **ultimate**: Hybrid + ColBERT MaxSim

Use `rerank=True` for cross-encoder reranking on any mode.

In [5]:
# Compare modes on hybrid tier
query = "learning algorithms"

print(f"Query: '{query}'\n")

# Dense mode
results_dense = db_hybrid.search(query, mode="dense")
print("Dense mode:")
print(f"  Top: {results_dense.top.text[:60]}...\n")

# Sparse mode
results_sparse = db_hybrid.search(query, mode="sparse")
print("Sparse mode:")
print(f"  Top: {results_sparse.top.text[:60]}...\n")

# Hybrid mode
results_hybrid = db_hybrid.search(query, mode="hybrid")
print("Hybrid mode:")
print(f"  Top: {results_hybrid.top.text[:60]}...")

Query: 'learning algorithms'



Dense mode:
  Top: Deep learning uses neural networks with many layers....



Sparse mode:
  Top: Reinforcement learning trains agents through rewards....



Hybrid mode:
  Top: Reinforcement learning trains agents through rewards....


## Reranking

Use `rerank="cross-encoder"` for higher quality results.

In [6]:
# Rerank uses a cross-encoder to reorder results
results_rerank = db_hybrid.search("image recognition", rerank="cross-encoder")

print("Reranked results:")
for r in results_rerank:
    print(f"  {r.score:.4f}: {r.text[:50]}...")

Reranked results:
  -9.1190: Reinforcement learning trains agents through rewar...
  -9.1190: Computer vision enables machines to interpret imag...
  -9.1190: Deep learning uses neural networks with many layer...
  -9.1190: Natural language processing handles text and speec...
  -9.1190: Machine learning is a subset of artificial intelli...


## Cleanup

In [7]:
import shutil, os

for folder in ["test_dense", "test_hybrid", "test_ultimate"]:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Deleted {folder}")